In [1]:
from landmark_detection.utils import load_image_database
import os
import onnxruntime as ort
import cv2
import torch
import numpy as np
import time
from tqdm import tqdm

In [2]:
versions = [51, 52, 53, 54, 55, 56, 57, 58, 59]
avg_times_per_version = {}  # Guardamos los tiempos promedio aquí

for version in versions:
    print(f"\n🔄 Ejecutando versión {version}...")

    # Paths
    PIPELINE_ONNX = os.path.join("landmark_detection", "models", f"pipeline-yolo-cvnet-sg-v{version}.onnx")
    RESULTS_PATH = os.path.join("datasets", "roxford5k", "results", f"df_roxford5k-v{version}.pkl")
    DESCRIPTORS_PATH = os.path.join("datasets", "roxford5k", "results", f"desc_roxford5k-v{version}.pkl")
    IMAGE_PATH = os.path.join("datasets", "roxford5k", "jpg")
    
    # Cargar modelo y datos
    pipeline = ort.InferenceSession(PIPELINE_ONNX, providers=["CPUExecutionProvider"])
    df_result, descriptors_final, places_db = load_image_database(RESULTS_PATH, DESCRIPTORS_PATH)
    image_names = df_result[df_result['class_id'] == -1]['image_name']
    
    # Convertir base de datos a numpy
    if isinstance(places_db, torch.Tensor):
        db_np = places_db.detach().cpu().numpy().astype(np.float32)
    else:
        db_np = np.asarray(places_db, dtype=np.float32)

    # Procesar imágenes con barra de progreso
    total_time = 0
    for image_name in tqdm(image_names, desc=f"Procesando v{version}", unit="img"):
        image = os.path.join(IMAGE_PATH, image_name)
        img_bgr = cv2.imread(image)
        img_tensor = torch.as_tensor(img_bgr)

        pipeline_inputs = {
            pipeline.get_inputs()[0].name: img_tensor.numpy(),
            pipeline.get_inputs()[1].name: db_np,
        }

        start = time.time()
        pipeline.run(None, pipeline_inputs)
        end = time.time()

        total_time += end - start

    avg_time = total_time / len(image_names)
    avg_times_per_version[version] = avg_time
    print(f"⏱️ Tiempo promedio por imagen (v{version}): {avg_time:.4f} segundos")

# Mostrar todos los resultados
print("\n📊 Tiempos promedio por versión:")
for version, avg_time in avg_times_per_version.items():
    print(f"  - Versión {version}: {avg_time:.4f} s/img")


🔄 Ejecutando versión 51...


Procesando v51: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [51:47<00:00,  1.63img/s]


⏱️ Tiempo promedio por imagen (v51): 0.5655 segundos

🔄 Ejecutando versión 52...


Procesando v52: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [58:56<00:00,  1.43img/s]


⏱️ Tiempo promedio por imagen (v52): 0.6474 segundos

🔄 Ejecutando versión 53...


...

⏱️ Tiempo promedio por imagen (v57): 1.0174 segundos

🔄 Ejecutando versión 58...


Procesando v58: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [1:39:50<00:00,  1.18s/img]


⏱️ Tiempo promedio por imagen (v58): 1.1315 segundos

🔄 Ejecutando versión 59...


Procesando v59: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [2:11:01<00:00,  1.55s/img]

⏱️ Tiempo promedio por imagen (v59): 1.5018 segundos

📊 Tiempos promedio por versión:
  - Versión 51: 0.5655 s/img
  - Versión 52: 0.6474 s/img
  - Versión 53: 0.7018 s/img
  - Versión 54: 0.7638 s/img
  - Versión 55: 0.8489 s/img
  - Versión 56: 0.9257 s/img
  - Versión 57: 1.0174 s/img
  - Versión 58: 1.1315 s/img
  - Versión 59: 1.5018 s/img
